# Introduction

Download financial data of the stocks in OMXSPI (390ish stocks listed in Stockholm) and rank them based on the financial metric EV / EBIT

IMPORTANT: I have implemented some measures to avoid rate limitations when downloading data. But, the block labelled 'ev / ebit calculation' will likely have to be run multiple times due to Yahoo Finance FREE API limiting access.

The partial results are saved as pkl files each time so it finishes eventually without having to do the same work multiple times.

January 2025.

# Download data.

In [5]:
# download ticker data for OMXSPI from Nasdaq OMX

# example address:
# 'https://indexes.nasdaqomx.com/Index/ExportWeightings/OMXSPI?tradeDate=2025-01-16T00:00:00.000&timeOfDay=SOD'

# need to replace '2025-01-16' with todays date in yyyy-mm-dd format.
# this ensures that I always have the latest index-components.

from datetime import date

# Get today's date
today = date.today()
print("Today's date:", today)

# string1 is static part of the HTTP adress
# string 2 is dynamically updated.

string1 = 'https://indexes.nasdaqomx.com/Index/ExportWeightings/OMXSPI?tradeDate='
string2 = str(today) + 'T00:00:00.000&timeOfDay=SOD'
http = string1 + string2

# the '{http}' makes sure the variable http is being treated as a string.
# this make sure there are no problems with special characters.

!wget -O tickers.xlsx '{http}'

Today's date: 2025-04-25
--2025-04-25 12:48:45--  https://indexes.nasdaqomx.com/Index/ExportWeightings/OMXSPI?tradeDate=2025-04-25T00:00:00.000&timeOfDay=SOD
Resolving indexes.nasdaqomx.com (indexes.nasdaqomx.com)... 45.60.150.18
Connecting to indexes.nasdaqomx.com (indexes.nasdaqomx.com)|45.60.150.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21446 (21K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘tickers.xlsx’

tickers.xlsx        100%[===================>]  20.94K  --.-KB/s    in 0.08s   

2025-04-25 12:48:47 (276 KB/s) - ‘tickers.xlsx’ saved [21446/21446]



In [6]:
import pandas as pd
import numpy as np

# Read the ticker file into a Pandas DataFrame
df = pd.read_excel("tickers.xlsx", engine="openpyxl")
df = df.dropna()

In [7]:
tickers = tickers = df['Unnamed: 1'].iloc[1:]   #.iloc[1:] removes first row
tickers_l = tickers.tolist() # create list of tickets.
print(f' 5 first components of tickers_l: {tickers_l[:5]}')

 5 first components of tickers_l: ['8TRA', 'AAK', 'ABB', 'ACAD', 'ACE']


In [8]:
# Go from the Security Symbols used by Nasdaq to yahoo finance tickers.
# this is necessary because the data is being fetched from Yahoo Finance.

def convert_to_yahoo_ticker(name):
    # Replace spaces with dashes and append .ST for Stockholm stocks
    ticker = name.replace(" ", "-") + ".ST"
    return ticker

# Example usage
#companies = ["XANO B", "VOLVO B", "ERIC B"]
#yahoo_tickers = [convert_to_yahoo_ticker(company) for company in companies]

In [9]:
yahoo_tickers = [convert_to_yahoo_ticker(company) for company in tickers_l]
yahoo_tickers[:5]

['8TRA.ST', 'AAK.ST', 'ABB.ST', 'ACAD.ST', 'ACE.ST']

In [10]:
# Remove the A entry if both A and B stocks are in the list.
# for Example, we don't need 'ACRI-A.ST', and 'ACRI-B.ST', get rid of A.

# A is usually less liquid so that's the one to get rid of.
# Some exceptions exist. SHB B is for example less liquid than
# SHB A. But, in general A is the one to get rid of.

# Set to keep track of the stocks we want to keep
tickers_list = set()

# Iterate over each ticker
for ticker in yahoo_tickers:
    # Check if it's a version A (contains '-A') and see if its B counterpart exists
    if '-A' in ticker:
        counterpart = ticker.replace('-A', '-B')
        if counterpart in yahoo_tickers:
            continue  # Skip the A version if the B version exists
    # Add to the set if it's not an A version or no B counterpart exists
    tickers_list.add(ticker)

# Convert the set back to a list (optional)
tickers_list = list(tickers_list)

print(tickers_list)
print(len(tickers_list))

['SKIS-B.ST', 'INTEA-D.ST', 'BOOZT.ST', 'NETEL.ST', 'SEDANA.ST', 'RATO-B.ST', 'SHOT.ST', 'HAKI-B.ST', 'SF.ST', 'ACTI.ST', 'CLAS-B.ST', 'DORO.ST', 'IAR-B.ST', 'ANOD-B.ST', 'AFRY.ST', 'RAIL.ST', 'HEM.ST', 'HM-B.ST', 'SINCH.ST', 'EPRO-B.ST', 'ARISE.ST', 'LIFCO-B.ST', 'SOF-B.ST', 'EPIS-B.ST', 'ORX.ST', 'HNSA.ST', 'NICA.ST', 'BALCO.ST', 'LIAB.ST', 'SECU-B.ST', 'INDU-C.ST', 'VIMIAN.ST', 'HANZA.ST', 'TELIA.ST', 'HOFI.ST', 'MMGR-B.ST', 'KARNEL-B.ST', 'MTG-B.ST', 'EQL.ST', 'HEXA-B.ST', 'MAHA-A.ST', 'ORES.ST', 'INFREA.ST', 'VITR.ST', 'EWRK.ST', 'EG7.ST', 'IS.ST', 'SEAF.ST', 'ALLEI.ST', 'KDEV.ST', 'BEIJ-B.ST', 'PRIC-B.ST', 'TFBANK.ST', 'NORB-B.ST', 'BILL.ST', 'ACRI-B.ST', 'QLIRO.ST', 'PREC.ST', 'CNCJO-B.ST', 'REJL-B.ST', 'GETI-B.ST', 'ASSA-B.ST', 'STAR-B.ST', 'BILI-A.ST', 'FOI-B.ST', 'HUMBLE.ST', 'CAT-B.ST', 'BETCO.ST', 'KLARA-B.ST', 'BERNER-B.ST', 'G5EN.ST', 'CTEK.ST', 'TEL2-B.ST', 'NCAB.ST', 'BOL.ST', 'INTEA-B.ST', 'BORG.ST', 'G2M.ST', 'PEAB-B.ST', 'SFAB.ST', 'PIERCE.ST', 'NOTE.ST', 'EVO.ST', '

# Build Database

Build SQLite database of ticker info and financial info. This will be used
for calculations later.

In [11]:
# Helper functions for the next block that builds the database from financal info


def ticker_already_done(ticker):
    """
    Function to check if data from a ticker has already been fetched
    This is used to help resume database-building after rate-limitation cut-off.
    """

    cur.execute("SELECT status FROM valuations WHERE ticker = ?", (ticker,))
    row = cur.fetchone()
    return row is not None and row[0] == 'ok'

import pandas as pd

def get_ebit(company):

    try:
      ebit_series = company.income_stmt.loc['EBIT']
      ebit = ebit_series.iloc[0]
    except KeyError:
        ebit = None

    return ebit


import pandas as pd

def get_net_debt(company, debt_fields):
    """
    Function to calculate net debt for a given company.

    Args:
    - company: yfinance Ticker object representing the company.
    - debt_fields: List of potential debt fields to calculate net debt.

    Returns:
    - net_debt: Calculated net debt value or None if no valid data found.
    """
    net_debt = None

    # Try to get 'Net Debt' first
    try:
        net_debt_series = company.balance_sheet.loc['Net Debt']
        net_debt = net_debt_series.iloc[0]
    except KeyError:
        pass  # If 'Net Debt' is not available, we continue

    # If 'Net Debt' is missing or NaN, try to calculate it from other debt fields
    if pd.isna(net_debt):  # Check if it's NaN
        for debt_field in debt_fields:
            try:
                # Get the total debt for the current debt field
                total_debt_series = company.balance_sheet.loc[debt_field]
                total_debt = total_debt_series.iloc[0]

                # Get cash and equivalents
                cash_series = company.balance_sheet.loc['Cash And Cash Equivalents']
                cash = cash_series.iloc[0]

                # If either total_debt or cash is missing, skip this field
                if pd.isna(total_debt) or pd.isna(cash):
                    continue

                net_debt = total_debt - cash
                break  # Exit the loop once net_debt is calculated

            except KeyError:
                continue  # Skip if the field is not present

    # Return None if net_debt is still not available, otherwise return the calculated net_debt
    return None if pd.isna(net_debt) else net_debt


In [12]:
# This block: fetch stock data and build database.

# Given that I am using free API, this block will, most likely, get ratelimited.
# I have sleep timer in place to avoid fetching a lot of data too fast but
# that might not be enough.

# the function ticker_already_done keeps track of what has already been
# processed so that this block can be ran multiple times til completition.


import sqlite3
import yfinance as yf
import time

conn = sqlite3.connect('valuations.db')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS valuations (
    ticker TEXT PRIMARY KEY,
    sector TEXT,
    market_cap REAL,
    net_debt REAL,
    ebit REAL,
    status TEXT,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')
conn.commit()


# List of potential debt fields
# This is used by get_net_debt()
debt_fields = ['Net Debt', 'Total Debt', 'Total Liabilities Net Minority Interest']


for ticker in tickers_list:
    if ticker_already_done(ticker):
        print(f"Skipping {ticker} — already processed.")
        continue

    try:
        company = yf.Ticker(ticker)


        # if not found, return None is what the second argument does.

        MC = company.info.get('marketCap', None)
        sector = company.info.get('sector', None)

        ebit = get_ebit(company)


        # custom function as net_debt has complicated logic.
        net_debt = get_net_debt(company, debt_fields)


        insert_stmt = (
        "INSERT OR REPLACE INTO valuations (ticker, sector, market_cap, net_debt, ebit, status)"
        "values (?, ?, ?, ?, ?, ?)" #placeholder.
        )

        data = (ticker, sector, MC, net_debt, ebit, 'ok')
        cur.execute(insert_stmt, data)


    except Exception as e:
        print(f"Error with {ticker}: {e}")
        cur.execute("INSERT OR REPLACE INTO valuations (ticker, status) VALUES (?, ?)", (ticker, 'api_error'))

    conn.commit()
    time.sleep(2)  # avoid rate limits

In [13]:
# inspect the database



import sqlite3

# Connect to the database
conn = sqlite3.connect('valuations.db')
cur = conn.cursor()

# Execute a query to fetch all rows from the valuations table
cur.execute("SELECT * FROM valuations")

# Fetch all rows and print them
rows = cur.fetchall()

# Print each row
for row in rows:
    print(row)

# Close the connection when done
conn.close()


('SKIS-B.ST', 'Consumer Cyclical', 13073132544.0, 1894793000.0, 720272000.0, 'ok', '2025-04-25 12:48:49')
('INTEA-D.ST', 'Real Estate', 13637094400.0, 11978000000.0, 1141000000.0, 'ok', '2025-04-25 12:48:52')
('BOOZT.ST', 'Consumer Cyclical', 5396581376.0, -296000000.0, 472000000.0, 'ok', '2025-04-25 12:48:54')
('NETEL.ST', 'Industrials', 488029728.0, 746000000.0, 145000000.0, 'ok', '2025-04-25 12:48:57')
('SEDANA.ST', 'Healthcare', 1029131264.0, -188043000.0, -50766000.0, 'ok', '2025-04-25 12:49:00')
('RATO-B.ST', 'Industrials', 10657991680.0, 6820000000.0, 1589000000.0, 'ok', '2025-04-25 12:49:02')
('SHOT.ST', 'Consumer Cyclical', 15930156032.0, 128000000.0, 2873000000.0, 'ok', '2025-04-25 12:49:05')
('HAKI-B.ST', 'Industrials', 667955968.0, 370000000.0, 70000000.0, 'ok', '2025-04-25 12:49:08')
('SF.ST', 'Communication Services', 2268007168.0, 3936000000.0, -6919000000.0, 'ok', '2025-04-25 12:49:10')
('ACTI.ST', 'Healthcare', 107885032.0, -16270000.0, -39900000.0, 'ok', '2025-04-25 1

In [14]:
# create a second database that maps tickers to names

import sqlite3

conn = sqlite3.connect('ticker_mappings.db')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS ticker_mappings (
    ticker TEXT PRIMARY KEY,
    name TEXT
)
''')
conn.commit()

# create ticker --> company name mapping

# Create dictionary from col1 as keys and col2 as values
mapping_dict = dict(zip(df['Unnamed: 1'].iloc[1:], df['Unnamed: 0'].iloc[1:]))

# convert to yahoo keys.

# Create a new dictionary with modified keys
yahoo_mapping_dict = {convert_to_yahoo_ticker(key): value for key, value in mapping_dict.items()}

# inser the dict values into the database

for ticker, name in yahoo_mapping_dict.items():
    try:
        insert_stmt = '''
        INSERT OR REPLACE INTO ticker_mappings (ticker, name)
        VALUES (?, ?)
        '''
        cur.execute(insert_stmt, (ticker, name))
    except Exception as e:
        print(f"Error inserting {ticker}: {e}")

# Commit the changes and close the connection
conn.commit()
conn.close()

In [21]:
# Perform EV / EBIT ranking.

import sqlite3

"""
Table: valuations (
    ticker TEXT PRIMARY KEY,
    sector TEXT,
    market_cap REAL,
    net_debt REAL,
    ebit REAL,
    status TEXT,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
"""

"""
Table: ticker_mappings (
    ticker TEXT PRIMARY KEY,
    name TEXT
)
"""

# Connect to the database
conn = sqlite3.connect('valuations.db')
cur = conn.cursor()

# Attach the second database (ticker_mappings.db) to the current connection
cur.execute("ATTACH DATABASE 'ticker_mappings.db' AS ticker_mappings")


# this query:
# calculate ev / ebit rounded to two decimals
# exclude Real Estate and Financials
# join valuations to ticker_mappings so I can display names instead of tickers.
# exclude negative EV / EBIT
# rank by lowest positive EV / EBIT (Best)

cur.execute("""
SELECT
t.name,
ROUND(
(v.market_cap + v.net_debt) / v.ebit
,2) AS ev_ebit
FROM valuations v
JOIN ticker_mappings t
ON t.ticker = v.ticker
WHERE
v.sector NOT IN ('Real Estate', 'Financial Services')
AND
v.ebit > 0
ORDER BY ev_ebit ASC
""")


# Fetch all rows
rows = cur.fetchall()

# Print top {fraction}% companies ranked by EV / EBIT

fraction = 0.20
length = len(rows)
n = int(fraction * length)

print(f"Top {n} companies ranked by EV / EBIT \n")

for row in rows[:n]:
    print(f"Company: {row[0]}, EV / EBIT: {row[1]}")

# Close the connection when done
conn.close()

Top 43 companies ranked by EV / EBIT 

Company: Volvo Car AB ser. B, EV / EBIT: 1.47
Company: Eniro Group AB, EV / EBIT: 2.2
Company: SSAB AB ser. B, EV / EBIT: 2.53
Company: Saniona AB, EV / EBIT: 2.83
Company: Enea AB, EV / EBIT: 3.03
Company: Dedicare AB ser. B, EV / EBIT: 4.8
Company: Bong AB, EV / EBIT: 5.01
Company: Pricer AB ser. B, EV / EBIT: 5.27
Company: Scandic Hotels Group AB, EV / EBIT: 5.59
Company: AcadeMedia AB, EV / EBIT: 6.07
Company: G5 Entertainment AB, EV / EBIT: 6.13
Company: Bulten AB, EV / EBIT: 6.44
Company: ProfilGruppen AB ser. B, EV / EBIT: 6.61
Company: Nordic Paper Holding AB, EV / EBIT: 6.65
Company: Boliden AB, EV / EBIT: 6.78
Company: NOVOTEK AB ser. B, EV / EBIT: 6.87
Company: B3 Consulting Group AB, EV / EBIT: 6.88
Company: Arctic Paper S.A., EV / EBIT: 6.94
Company: Sleep Cycle AB, EV / EBIT: 7.05
Company: Profoto Holding AB, EV / EBIT: 7.11
Company: Elanders AB ser. B, EV / EBIT: 7.54
Company: Humana AB, EV / EBIT: 7.55
Company: Green Landscaping Gr